## Multi class Classification

In [10]:
import keras
import tensorflow as tf
import sys
import sklearn as sk
import pandas as pd

print("Tensor Flow Version: {}".format(tf.__version__))
print("Keras Version: {}".format(keras.__version__))
print()
print("Python {}".format(sys.version))
print('Pandas {}'.format(pd.__version__))
print('Scikit-Learn {}'.format(sk.__version__))

Tensor Flow Version: 1.10.0
Keras Version: 2.2.2

Python 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas 0.23.0
Scikit-Learn 0.19.1


In [11]:
import pandas as pd
from sklearn import preprocessing

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)

    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type

    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [3]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

path = "C:/Users/Ayra/Desktop"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename)
df.head()

C:\Users\Ayra\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [15]:
species = encode_text_index(df,"species")
x,y = to_xy(df,"species")
print("x:",x[:5])
print("y:",y[:5])

x: [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
y: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


C:\Users\Ayra\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [16]:
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(20, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=0, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=0,epochs=1000)
model.load_weights('best_weights.hdf5')

In [23]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import figure, show

from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix

pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_test2 = np.argmax(y_test,axis=1)
    
# Compute confusion matrix
cm = confusion_matrix(y_test2, pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, species)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
#plot_confusion_matrix(cm_normalized, species, title='Normalized confusion matrix')

plt.show()

Confusion matrix, without normalization
[[15  0  0]
 [ 0 11  0]
 [ 0  0 12]]


NameError: name 'plt_confusion_matrix' is not defined

<Figure size 432x288 with 0 Axes>

In [3]:
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)

    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type

    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

path = "C:/Users/Ayra/Desktop"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename)
df.head()

species = encode_text_index(df,"species")
x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=45)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

C:\Users\Ayra\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 6         
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________
Train on 150 samples, validate on 38 samples
Epoch 1/1000
 - 0s - loss: 1.0970 - val_loss: 1.0942
Epoch 2/1000
 - 0s - loss: 1.0926 - val_loss: 1.0897
Epoch 3/1000
 - 0s - loss: 1.0880 - val_loss: 1.0864
Epoch 4/1000
 - 0s - loss: 1.0847 - val_loss: 1.0830
Epoch 5/1000
 - 0s - loss: 1.0809 - val_loss: 1.0792
Epoch 6/1000
 - 0s - loss: 1.0771 - val_loss: 1.0750
Epoch 7/1000
 - 0s - loss: 1.0727 - val_loss: 1

Epoch 138/1000
 - 0s - loss: 0.3612 - val_loss: 0.3174
Epoch 139/1000
 - 0s - loss: 0.3600 - val_loss: 0.3156
Epoch 140/1000
 - 0s - loss: 0.3586 - val_loss: 0.3142
Epoch 141/1000
 - 0s - loss: 0.3579 - val_loss: 0.3124
Epoch 142/1000
 - 0s - loss: 0.3563 - val_loss: 0.3122
Epoch 143/1000
 - 0s - loss: 0.3552 - val_loss: 0.3112
Epoch 144/1000
 - 0s - loss: 0.3539 - val_loss: 0.3111
Epoch 145/1000
 - 0s - loss: 0.3527 - val_loss: 0.3099
Epoch 146/1000
 - 0s - loss: 0.3514 - val_loss: 0.3087
Epoch 147/1000
 - 0s - loss: 0.3503 - val_loss: 0.3074
Epoch 148/1000
 - 0s - loss: 0.3490 - val_loss: 0.3059
Epoch 149/1000
 - 0s - loss: 0.3477 - val_loss: 0.3051
Epoch 150/1000
 - 0s - loss: 0.3466 - val_loss: 0.3038
Epoch 151/1000
 - 0s - loss: 0.3454 - val_loss: 0.3031
Epoch 152/1000
 - 0s - loss: 0.3442 - val_loss: 0.3013
Epoch 153/1000
 - 0s - loss: 0.3428 - val_loss: 0.3001
Epoch 154/1000
 - 0s - loss: 0.3416 - val_loss: 0.2991
Epoch 155/1000
 - 0s - loss: 0.3406 - val_loss: 0.2985
Epoch 156/

Epoch 287/1000
 - 0s - loss: 0.2085 - val_loss: 0.1739
Epoch 288/1000
 - 0s - loss: 0.2077 - val_loss: 0.1733
Epoch 289/1000
 - 0s - loss: 0.2073 - val_loss: 0.1724
Epoch 290/1000
 - 0s - loss: 0.2064 - val_loss: 0.1718
Epoch 291/1000
 - 0s - loss: 0.2059 - val_loss: 0.1705
Epoch 292/1000
 - 0s - loss: 0.2049 - val_loss: 0.1702
Epoch 293/1000
 - 0s - loss: 0.2043 - val_loss: 0.1706
Epoch 294/1000
 - 0s - loss: 0.2038 - val_loss: 0.1705
Epoch 295/1000
 - 0s - loss: 0.2029 - val_loss: 0.1686
Epoch 296/1000
 - 0s - loss: 0.2024 - val_loss: 0.1672
Epoch 297/1000
 - 0s - loss: 0.2017 - val_loss: 0.1674
Epoch 298/1000
 - 0s - loss: 0.2014 - val_loss: 0.1664
Epoch 299/1000
 - 0s - loss: 0.2001 - val_loss: 0.1672
Epoch 300/1000
 - 0s - loss: 0.1998 - val_loss: 0.1673
Epoch 301/1000
 - 0s - loss: 0.1990 - val_loss: 0.1658
Epoch 302/1000
 - 0s - loss: 0.1999 - val_loss: 0.1635
Epoch 303/1000
 - 0s - loss: 0.1974 - val_loss: 0.1639
Epoch 304/1000
 - 0s - loss: 0.1969 - val_loss: 0.1647
Epoch 305/

In [4]:
np.set_printoptions(suppress=True)
pred = model.predict(x_test)
print(pred[0:10])

[[0.9993592  0.00064044 0.00000031]
 [0.9985115  0.00148729 0.00000121]
 [0.00025519 0.2594737  0.74027115]
 [0.9982969  0.00170151 0.00000151]
 [0.99889404 0.0011053  0.00000075]
 [0.99906    0.00093944 0.00000058]
 [0.99805486 0.00194327 0.00000187]
 [0.00000045 0.0279872  0.9720124 ]
 [0.00027982 0.26696667 0.7327535 ]
 [0.0000019  0.04765328 0.9523448 ]]


In [5]:
from sklearn.metrics import log_loss

# Using the predictions (pred) and the known 1-hot encodings (y_test) we can compute the log-loss error.  
# The lower a log loss the better.  The probabilities (pred) from the previous section specify how sure the neural network
# is of its prediction.  Log loss error pubishes the neural network (with a lower score) for very confident, but wrong,
# classifications.
print(log_loss(y_test,pred))

0.1524883397865587


In [7]:
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)

print("Predictions: {}".format(predict_classes))
print("Expected: {}".format(expected_classes))
print(species[predict_classes[1:10]])

Predictions: [0 0 2 0 0 0 0 2 2 2 0 2 2 2 2 0 2 2 0 1 1 1 2 1 0 2 1 1 0 1 1 1 2 2 0 2 0
 0]
Expected: [0 0 2 0 0 0 0 2 2 2 0 2 2 2 2 0 2 2 0 1 1 1 2 1 0 2 1 1 0 1 1 1 2 2 0 2 0
 0]
['Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-virginica' 'Iris-virginica' 'Iris-virginica']


In [8]:
from sklearn.metrics import accuracy_score

# Accuracy might be a more easily understood error metric.  It is essentially a test score.  For all of the iris predictions,
# what percent were correct?  The downside is it does not consider how confident the neural network was in each prediction.

correct = accuracy_score(expected_classes,predict_classes)
print("Accuracy: {}".format(correct))

Accuracy: 1.0
